# Dev Notebook

This notebook loads the version of trulens_eval from the enclosing repo folder. You can use this to debug or devlop trulens_eval features.

In [ ]:
# ! pip uninstall -y trulens_eval # ==0.18.0
# ! pip list | grep trulens

In [ ]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import sys

# If running from github repo, can use this:
sys.path.append(str(Path().cwd().parent.parent.resolve()))

# Enables: Debugging printouts.
"""
import logging
root = logging.getLogger()
root.setLevel(logging.DEBUG)

handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
root.addHandler(handler)
"""

In [ ]:
import os
for k in os.environ:
    if "KEY" in k:
        print(k)

In [ ]:
from trulens_eval import Bedrock
bedrock = Bedrock(model_engine="Bedrock", model_id = "amazon.titan-tg1-large", region_name="us-west-2")

In [ ]:
from trulens_eval.feedback.provider.endpoint.base import Endpoint

In [ ]:
Endpoint.instrumented_methods
Endpoint.print_instrumented()

In [ ]:
from trulens_eval import Huggingface
Huggingface()

In [ ]:
from concurrent.futures import as_completed
from time import sleep

# from examples.expositional.end2end_apps.custom_app.custom_app import CustomApp
from tqdm.auto import tqdm

from trulens_eval import Feedback
from trulens_eval import Tru
from trulens_eval.feedback.provider.hugs import Dummy
from trulens_eval.schema import FeedbackMode
from trulens_eval.tru_custom_app import TruCustomApp
from trulens_eval.utils.threading import TP

tp = TP()

d = Dummy(
    loading_prob=0.1,
    freeze_prob=0.0, # we expect requests to have their own timeouts so freeze should never happen
    error_prob=0.01,
    overloaded_prob=0.1,
    rpm=6000
)

tru = Tru()

tru.reset_database()

tru.start_dashboard(
    force = True,
    # _dev=Path().cwd().parent.parent.resolve()
)

In [ ]:
from boto3 import client
c1 = client(service_name="bedrock-runtime")
c2 = client(service_name="bedrock-runtime")

In [ ]:
from botocore.client import ClientCreator

In [ ]:
import boto3

In [ ]:
dir(boto3.utils)

In [ ]:
import botocore
dir(botocore.client)

In [ ]:
from IPython.display import JSON

# Imports main tools:
from trulens_eval import TruChain, Feedback, Huggingface, Tru
from trulens_eval.schema import FeedbackResult
tru = Tru()

# Imports from langchain to build app. You may need to install langchain first
# with the following:
# ! pip install langchain>=0.0.170
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.prompts import HumanMessagePromptTemplate

# Initialize Huggingface-based feedback function collection class:
# bedrock = Bedrock(model_engine="Bedrock", model_id = "anthropic.claude-v2", region_name="us-west-2")

# Define a language match feedback function using HuggingFace.
f_relevance = Feedback(bedrock.relevance).on_input_output()
# By default this will check language match on the main app input and main app
# output.

full_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        template=
        "Provide a helpful response with relevant background information for the following: {prompt}",
        input_variables=["prompt"],
    )
)

chat_prompt_template = ChatPromptTemplate.from_messages([full_prompt])

llm = OpenAI(temperature=0.9, max_tokens=128)

chain = LLMChain(llm=llm, prompt=chat_prompt_template, verbose=True)

tru_recorder = TruChain(
    chain,
    app_id='Chain1_ChatApplication',
    feedbacks=[f_relevance]
)

with tru_recorder as recording:
    llm_response = chain.run("What's the capital of the USA?")

display(llm_response)

In [ ]:
import os
import dotenv
from pathlib import Path

from trulens_eval import Tru, TruBasicApp, Feedback
from trulens_eval.schema import Cost, Record

SCRIPT_DIR = Path().cwd()
dotenv.load_dotenv(SCRIPT_DIR / "my.env")

tru = Tru(database_redact_keys=True)#database_url=os.environ.get("database_url"))

def llm_standalone(prompt):
    return prompt

f_sentiment = Feedback(bedrock.sentiment).on_output()

recorder = TruBasicApp(llm_standalone, app="default", feedbacks=[f_sentiment])

In [ ]:
with recorder as recording:
    response = recorder.app("This sucks")

print(response)

In [ ]:
with recorder as recording:
    response = recorder.app("This rocks")

print(response)

In [ ]:
rec = recording.get()

In [ ]:
rec

In [ ]:
rec.dict()

In [ ]:
recorder.dict()